## Data Acquisition

First I will use the `sodapy` package to set up a data stream using the Open Data API. This data is updated monthly, and so by using an API, the data will always be up-to-date.

In [1]:
import sodapy

In [ ]:
from sodapy import Socrata

In [ ]:
bike_count_end = 'https://data.cityofnewyork.us/resource/uczf-rk3c.json'
bike_counter_end = 'https://data.cityofnewyork.us/resource/smn3-rzf9.json'